In [53]:
! pip install python-dotenv langchain langchain-community langchainhub openai tiktoken azure-ai-documentintelligence azure-identity azure-search-documents==11.4.0b8
! pip install langchain --upgrade

5762.63s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


5768.88s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [54]:
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.
os.environ["openai.api_type"] = os.getenv("openai.api_type")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
doc_intelligence_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")
api_version = os.getenv("openai.api_version")
ada_deployed_model = os.getenv("ada")
gpt4_deployed_model = os.getenv("gpt4")


In [55]:

from langchain import hub
from langchain.chat_models import AzureChatOpenAI
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch

In [56]:
# Initiate Azure AI Document Intelligence to load the document. You can either specify file_path or url_path to load the document.
loader = AzureAIDocumentIntelligenceLoader(
    file_path="../sample_docs/pdf-sample.pdf",
    api_key=doc_intelligence_key,
    api_endpoint=doc_intelligence_endpoint,
    api_model="prebuilt-layout",
)
docs = loader.load()

# Split the document into chunks base on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

docs_string = docs[0].page_content
splits = text_splitter.split_text(docs_string)

print("Length of splits: " + str(len(splits)))

Length of splits: 1


In [57]:
# Embed the splitted documents and insert into Azure Search vector store
# openai.api_base = os.getenv("openai.api_base")
import openai 
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")

aoai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment= ada_deployed_model,
    openai_api_version= api_version,  # e.g., "2023-07-01-preview"
)

vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")

index_name: str = "index11"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=aoai_embeddings.embed_query,
)

vector_store.add_documents(documents=splits)

['Yzk5OTZjMTctOTNjMS00NjU3LWFhMDktNTJiMTBkNmFiNjBm']

In [58]:
# Retrieve relevant chunks based on the question

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.get_relevant_documents(
    "<your question>"
)

print(retrieved_docs[0].page_content)

# Use a prompt for RAG that is checked into the LangChain prompt hub (https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=989ad331-949f-4bac-9694-660074a208a7)
prompt = hub.pull("rlm/rag-prompt")
llm = AzureChatOpenAI(
    openai_api_version=api_version,  # e.g., "2023-07-01-preview"
    azure_deployment=gpt4_deployed_model,
    temperature=0,
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

IndexError: list index out of range

In [ ]:
# Ask a question about the document

rag_chain.invoke("Who can see a pdf?")

'Anyone, anywhere can open a PDF file as long as they have the free Adobe Acrobat Reader. This software is easy to download and can be freely distributed by anyone. The PDF format is universal and preserves all of the fonts, formatting, colours and graphics of any source document.'